pip install prophet --quiet - встановіть через термінал.
pip install pandas
pip install matplotlib

Завантаження, та підготовка даних.

In [2]:
import pandas as pd
from prophet import Prophet

In [3]:
df = pd.read_csv('forecast_revenue_dataset.csv')

In [5]:
# Show first few rows
df.head(10)

,date,category_id,category_title,is_sale_prohibition,year,cos_month,sin_month,cos_quarter,sin_quarter,unique_brand_count,revenue
0,2018-01-01,1,Wine,0.0,2018,0.866,0.500,0,1,249.0,1.558012e+08
1,2018-01-01,2,Champagne & Sparkling Wine,0.0,2018,0.866,0.500,0,1,86.0,2.185618e+08
2,2018-01-01,3,Whiskey,0.0,2018,0.866,0.500,0,1,80.0,5.846833e+08
3,2018-01-01,4,Cognac & Brandy,0.0,2018,0.866,0.500,0,1,51.0,9.918995e+07
4,2018-01-01,5,Vodka,0.0,2018,0.866,0.500,0,1,51.0,1.079853e+08
5,2018-02-01,1,Wine,0.0,2018,0.500,0.866,0,1,272.0,1.438317e+08
6,2018-02-01,2,Champagne & Sparkling Wine,0.0,2018,0.500,0.866,0,1,97.0,1.832839e+08
7,2018-02-01,3,Whiskey,0.0,2018,0.500,0.866,0,1,85.0,6.649445e+08
8,2018-02-01,4,Cognac & Brandy,0.0,2018,0.500,0.866,0,1,59.0,1.171972e+08
9,2018-02-01,5,Vodka,0.0,2018,0.500,0.866,0,1,54.0,9.390119e+07


In [6]:
# Step 4: Convert 'date' to datetime
df['date'] = pd.to_datetime(df['date'])

Запуск моделі.

In [7]:
money_results = []

In [8]:
for cat in df['category_id'].unique():
    cat_df = df[df['category_id'] == cat][['date', 'revenue']].copy()

    # Rename columns for Prophet
    cat_df.rename(columns={'date': 'ds', 'revenue': 'y'}, inplace=True)

    # Skip categories with too little data
    if len(cat_df) < 6:
        continue

    # Initialize and fit model
    model = Prophet()
    model.fit(cat_df)

    # Make future dataframe for 12 months ahead
    future = model.make_future_dataframe(periods=12, freq='MS')  # MS = Month Start

    # Predict
    forecast = model.predict(future)

    # Extract forecast for future 12 months
    forecast_tail = forecast[['ds', 'yhat']].tail(12).copy()
    forecast_tail.rename(columns={'ds': 'date', 'yhat': 'forecast_revenue'}, inplace=True)
    forecast_tail['category_id'] = cat

    money_results.append(forecast_tail)

21:21:44 - cmdstanpy - INFO - Chain [1] start processing
21:21:44 - cmdstanpy - INFO - Chain [1] done processing
21:21:44 - cmdstanpy - INFO - Chain [1] start processing
21:21:44 - cmdstanpy - INFO - Chain [1] done processing
21:21:44 - cmdstanpy - INFO - Chain [1] start processing
21:21:44 - cmdstanpy - INFO - Chain [1] done processing
21:21:44 - cmdstanpy - INFO - Chain [1] start processing
21:21:44 - cmdstanpy - INFO - Chain [1] done processing
21:21:44 - cmdstanpy - INFO - Chain [1] start processing
21:21:44 - cmdstanpy - INFO - Chain [1] done processing


Збереження результатів.

In [9]:
# Step 7: Combine results and export
final_forecast = pd.concat(money_results, ignore_index=True)
final_forecast = final_forecast[['date', 'category_id', 'forecast_revenue']]
final_forecast.to_csv('forecast_revenue_by_category_12_months.csv', index=False)



In [10]:
# Optional: display first few rows
final_forecast.head()

,date,category_id,forecast_revenue
0,2026-09-01,1,1.603206e+08
1,2026-10-01,1,1.627739e+08
2,2026-11-01,1,1.436821e+08
3,2026-12-01,1,1.287619e+08
4,2027-01-01,1,1.448332e+08


In [11]:
print("all done ;)")

all done ;)
